# Homework 4: CIFAR-10
Applied Neural Networks

## Create a CNN to classify CIFAR-10
Use the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

You can utilize `keras.datasets.cifar10.load_data()` to import the dataset as `numpy ndarrays` or `tfds.load('cifar10')` to import the dataset as `tf.data.Dataset`.

**You must utilize transfer learning and data augmentation.** Your grade will depend on the quality of your best model. Discuss the different models you tried. *Include all model parameters, accuracy, a confusion matrix, and sample misclassified images.*

I suggest using data pipelines to avoid a RAM resource exhaustion error. They can be implemented fairly easily in one of two ways: 
  1. Utilize a data_augmentation function and Dataset.map(data_augmentation) to augment your training tf.data.Dataset (will require `tf.data.Dataset`s), or
  2. Utilize Keras ImageDataGenerator, using the ImageDataGenerator to also do all preprocessing necessary and batch images (will require `np ndarray`s).